In [16]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time
import matplotlib.pyplot as plt

In [34]:
sectors = {
    'Aerospace & Defence': ['00000085',1],
    'Banks': ['00000006',1],
    'Automobile': ['00000005',1],
    'Healthcare': ['00000030', 2],
    'Diamond, Gems and Jewellery': ['00000020',2],
    'Steel': ['00000057',3],
    'Plastic Products': ['00000048',2],
    'Pharmaceuticals': ['00000046',5],
    'Consumer Durables': ['00000017',2],
    'IT-Software': ['00000034',6]
}

In [38]:
headers = {
    "User-Agent": "Mozilla/5.0",
    "Cookie": "sessionid=n0coereufdfkzv675u1u50pognf7of6f"
}

col_names = []
for sector, (sector_code, total_pages) in sectors.items():
    all_rows = []

    for page in range(1, total_pages + 1):
        url = f'https://www.screener.in/company/compare/{sector_code}/?limit=50&page={page}'
        response = requests.get(url, headers=headers)
        time.sleep(1)
        soup = BeautifulSoup(response.text, "html.parser")

        print(f"{sector} - Page {page} status: {response.status_code}")

        if page == 1 and sector == 'Aerospace & Defence':
            col_group = soup.find_all(name="th", scope='colgroup')
            for col in col_group:
                for c in col:
                    text = c.getText().strip()
                    if text != "":
                        clean_text = re.sub(r'\s+', ' ', text)
                        if clean_text not in col_names:
                            col_names.append(clean_text)

        tds = soup.select("tr td")
        data = []
        for td in tds:
            a = td.find("a")
            if a:
                dta.append(a.getText().strip())a
            else:
                data.append(td.getText(strip=True))

        rows = [data[i:i+18] for i in range(0, len(data), 18)]
        all_rows.extend(rows)

    df = pd.DataFrame(all_rows, columns=col_names)
    df['Sector'] = sector
    df.to_csv(f"{sector}.csv", index=False)

Aerospace & Defence - Page 1 status: 200
Banks - Page 1 status: 200
Automobile - Page 1 status: 200
Healthcare - Page 1 status: 200
Healthcare - Page 2 status: 200
Diamond, Gems and Jewellery - Page 1 status: 200
Diamond, Gems and Jewellery - Page 2 status: 200
Steel - Page 1 status: 200
Steel - Page 2 status: 200
Steel - Page 3 status: 200
Plastic Products - Page 1 status: 200
Plastic Products - Page 2 status: 200
Pharmaceuticals - Page 1 status: 200
Pharmaceuticals - Page 2 status: 200
Pharmaceuticals - Page 3 status: 200
Pharmaceuticals - Page 4 status: 200
Pharmaceuticals - Page 5 status: 200
Consumer Durables - Page 1 status: 200
Consumer Durables - Page 2 status: 200
IT-Software - Page 1 status: 200
IT-Software - Page 2 status: 200
IT-Software - Page 3 status: 200
IT-Software - Page 4 status: 200
IT-Software - Page 5 status: 200
IT-Software - Page 6 status: 200


In [93]:
merged_df = pd.concat([pd.read_csv(f"{sector}.csv") for sector in sectors], ignore_index=True)
merged_df.to_csv("All_Sectors_Merged.csv", index=False)

In [94]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1007 entries, 0 to 1006
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   S.No.              1007 non-null   float64
 1   Name               1007 non-null   object 
 2   CMP Rs.            1005 non-null   float64
 3   Prom. Hold. %      1006 non-null   float64
 4   Debt Rs.Cr.        1004 non-null   float64
 5   NP Qtr Rs.Cr.      993 non-null    float64
 6   6mth return %      961 non-null    float64
 7   1Yr return %       918 non-null    float64
 8   3Yrs return %      782 non-null    float64
 9   All time high Rs.  1005 non-null   float64
 10  All time low Rs.   1005 non-null   float64
 11  P/E                837 non-null    float64
 12  EPS 12M Rs.        1000 non-null   float64
 13  Vol 1d             1007 non-null   int64  
 14  IV Rs.             777 non-null    float64
 15  PB X PE            795 non-null    float64
 16  CMP / BV           924 n

In [95]:
merged_df[merged_df["CMP Rs."].isnull()]

,S.No.,Name,CMP Rs.,Prom. Hold. %,Debt Rs.Cr.,NP Qtr Rs.Cr.,6mth return %,1Yr return %,3Yrs return %,All time high Rs.,All time low Rs.,P/E,EPS 12M Rs.,Vol 1d,IV Rs.,PB X PE,CMP / BV,Mar Cap Rs.Cr.,Sector
613,149.0,Accretion Pha.,NaN,NaN,13.48,NaN,NaN,NaN,NaN,NaN,NaN,75.35,3.72,0,NaN,NaN,NaN,112.27,Pharmaceuticals
857,111.0,Virtual Galaxy,NaN,87.95,38.64,NaN,NaN,NaN,NaN,NaN,NaN,21.55,14.90,0,NaN,NaN,NaN,353.13,IT-Software


In [6]:
# there is no CMP for these 2 rows and ALSO max columns are null values

In [97]:
merged_df = merged_df.dropna(subset=["CMP Rs."])

In [98]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1005 entries, 0 to 1006
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   S.No.              1005 non-null   float64
 1   Name               1005 non-null   object 
 2   CMP Rs.            1005 non-null   float64
 3   Prom. Hold. %      1005 non-null   float64
 4   Debt Rs.Cr.        1002 non-null   float64
 5   NP Qtr Rs.Cr.      993 non-null    float64
 6   6mth return %      961 non-null    float64
 7   1Yr return %       918 non-null    float64
 8   3Yrs return %      782 non-null    float64
 9   All time high Rs.  1005 non-null   float64
 10  All time low Rs.   1005 non-null   float64
 11  P/E                835 non-null    float64
 12  EPS 12M Rs.        998 non-null    float64
 13  Vol 1d             1005 non-null   int64  
 14  IV Rs.             777 non-null    float64
 15  PB X PE            795 non-null    float64
 16  CMP / BV           924 non-nu

In [99]:
# because there is no debt for the company (good company)
merged_df["Debt Rs.Cr."] = merged_df["Debt Rs.Cr."].fillna(0)

In [100]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1005 entries, 0 to 1006
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   S.No.              1005 non-null   float64
 1   Name               1005 non-null   object 
 2   CMP Rs.            1005 non-null   float64
 3   Prom. Hold. %      1005 non-null   float64
 4   Debt Rs.Cr.        1005 non-null   float64
 5   NP Qtr Rs.Cr.      993 non-null    float64
 6   6mth return %      961 non-null    float64
 7   1Yr return %       918 non-null    float64
 8   3Yrs return %      782 non-null    float64
 9   All time high Rs.  1005 non-null   float64
 10  All time low Rs.   1005 non-null   float64
 11  P/E                835 non-null    float64
 12  EPS 12M Rs.        998 non-null    float64
 13  Vol 1d             1005 non-null   int64  
 14  IV Rs.             777 non-null    float64
 15  PB X PE            795 non-null    float64
 16  CMP / BV           924 non-nu

In [101]:
merged_df[merged_df["NP Qtr Rs.Cr."].isnull()]

,S.No.,Name,CMP Rs.,Prom. Hold. %,Debt Rs.Cr.,NP Qtr Rs.Cr.,6mth return %,1Yr return %,3Yrs return %,All time high Rs.,All time low Rs.,P/E,EPS 12M Rs.,Vol 1d,IV Rs.,PB X PE,CMP / BV,Mar Cap Rs.Cr.,Sector
28,29.0,C2C Advanced,445.00,41.35,1.12,NaN,NaN,NaN,NaN,954.00,250.15,59.72,10.12,28200,NaN,NaN,NaN,740.54,Aerospace & Defence
94,24.0,Delta Auto.,71.00,70.73,3.87,NaN,NaN,NaN,NaN,183.75,66.50,15.87,221.93,17000,NaN,NaN,NaN,108.56,Automobile
130,35.0,Chandan Healthca,173.95,49.43,42.01,NaN,NaN,NaN,NaN,215.40,140.00,26.13,8.14,45600,NaN,NaN,NaN,425.34,Healthcare
203,26.0,Kabra Jewels,148.80,62.56,89.96,NaN,NaN,NaN,NaN,265.00,108.05,16.62,204.35,42000,NaN,NaN,NaN,156.02,"Diamond, Gems and Jewellery"
215,38.0,Manoj Jewellers,48.00,63.27,15.38,NaN,NaN,NaN,NaN,53.95,46.12,13.52,5.33,44000,NaN,NaN,NaN,43.13,"Diamond, Gems and Jewellery"
222,45.0,Kenrik Indust.,22.47,72.01,0.78,NaN,NaN,NaN,NaN,27.57,22.47,26.50,1.18,24000,NaN,NaN,NaN,28.09,"Diamond, Gems and Jewellery"
417,43.0,Srigee DLM,240.04,45.23,2.28,NaN,NaN,NaN,NaN,240.04,188.10,48.28,6.97,27600,NaN,NaN,NaN,143.39,Plastic Products
429,55.0,Indobell Insul.,115.20,65.00,5.27,NaN,NaN,NaN,NaN,166.75,83.03,70.46,2.52,1500,NaN,NaN,NaN,72.57,Plastic Products
430,56.0,Malpani Pipes,67.21,73.28,30.29,NaN,NaN,NaN,NaN,90.00,54.90,9.79,30.45,1600,NaN,NaN,NaN,72.44,Plastic Products
448,74.0,L.K.Mehta Poly.,55.80,72.92,6.48,NaN,NaN,NaN,NaN,71.10,47.90,24.92,13.76,1600,NaN,NaN,NaN,21.43,Plastic Products


### filling null values in EPS column

In [121]:
# Define a mask for rows where EPS is missing AND all required values are present
mask = (
    merged_df["EPS 12M Rs."].isnull() &
    merged_df["NP Qtr Rs.Cr."].notnull() &
    merged_df["Mar Cap Rs.Cr."].notnull() &
    merged_df["CMP Rs."].notnull()
)

# Apply the formula only to those rows
merged_df.loc[mask, "EPS 12M Rs."] = (
    merged_df.loc[mask, "NP Qtr Rs.Cr."] * 4 /
    (merged_df.loc[mask, "Mar Cap Rs.Cr."] / merged_df.loc[mask, "CMP Rs."])
)

In [124]:
merged_df[mask]

,S.No.,Name,CMP Rs.,Prom. Hold. %,Debt Rs.Cr.,NP Qtr Rs.Cr.,6mth return %,1Yr return %,3Yrs return %,All time high Rs.,All time low Rs.,P/E,EPS 12M Rs.,Vol 1d,IV Rs.,PB X PE,CMP / BV,Mar Cap Rs.Cr.,Sector


In [103]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1005 entries, 0 to 1006
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   S.No.              1005 non-null   float64
 1   Name               1005 non-null   object 
 2   CMP Rs.            1005 non-null   float64
 3   Prom. Hold. %      1005 non-null   float64
 4   Debt Rs.Cr.        1005 non-null   float64
 5   NP Qtr Rs.Cr.      993 non-null    float64
 6   6mth return %      961 non-null    float64
 7   1Yr return %       918 non-null    float64
 8   3Yrs return %      782 non-null    float64
 9   All time high Rs.  1005 non-null   float64
 10  All time low Rs.   1005 non-null   float64
 11  P/E                835 non-null    float64
 12  EPS 12M Rs.        1005 non-null   float64
 13  Vol 1d             1005 non-null   int64  
 14  IV Rs.             777 non-null    float64
 15  PB X PE            795 non-null    float64
 16  CMP / BV           924 non-nu

### fill P/E null values

In [104]:
merged_df.loc[
    merged_df["P/E"].isnull() &
    merged_df["CMP Rs."].notnull() &
    merged_df["EPS 12M Rs."].notnull(),
    "P/E"
] = merged_df["CMP Rs."] / merged_df["EPS 12M Rs."]


In [105]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1005 entries, 0 to 1006
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   S.No.              1005 non-null   float64
 1   Name               1005 non-null   object 
 2   CMP Rs.            1005 non-null   float64
 3   Prom. Hold. %      1005 non-null   float64
 4   Debt Rs.Cr.        1005 non-null   float64
 5   NP Qtr Rs.Cr.      993 non-null    float64
 6   6mth return %      961 non-null    float64
 7   1Yr return %       918 non-null    float64
 8   3Yrs return %      782 non-null    float64
 9   All time high Rs.  1005 non-null   float64
 10  All time low Rs.   1005 non-null   float64
 11  P/E                1005 non-null   float64
 12  EPS 12M Rs.        1005 non-null   float64
 13  Vol 1d             1005 non-null   int64  
 14  IV Rs.             777 non-null    float64
 15  PB X PE            795 non-null    float64
 16  CMP / BV           924 non-nu

### fill IV null values

In [106]:
merged_df.loc[
    merged_df["IV Rs."].isnull() &
    merged_df["EPS 12M Rs."].notnull() &
    merged_df["P/E"].notnull(),
    "IV Rs."
] = merged_df["EPS 12M Rs."] * merged_df["P/E"]


In [107]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1005 entries, 0 to 1006
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   S.No.              1005 non-null   float64
 1   Name               1005 non-null   object 
 2   CMP Rs.            1005 non-null   float64
 3   Prom. Hold. %      1005 non-null   float64
 4   Debt Rs.Cr.        1005 non-null   float64
 5   NP Qtr Rs.Cr.      993 non-null    float64
 6   6mth return %      961 non-null    float64
 7   1Yr return %       918 non-null    float64
 8   3Yrs return %      782 non-null    float64
 9   All time high Rs.  1005 non-null   float64
 10  All time low Rs.   1005 non-null   float64
 11  P/E                1005 non-null   float64
 12  EPS 12M Rs.        1005 non-null   float64
 13  Vol 1d             1005 non-null   int64  
 14  IV Rs.             1004 non-null   float64
 15  PB X PE            795 non-null    float64
 16  CMP / BV           924 non-nu

### droping a row where we cannot fill null value in IV column (which is only one row)

In [108]:
merged_df = merged_df.dropna(subset=["IV Rs."])

In [109]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1004 entries, 0 to 1006
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   S.No.              1004 non-null   float64
 1   Name               1004 non-null   object 
 2   CMP Rs.            1004 non-null   float64
 3   Prom. Hold. %      1004 non-null   float64
 4   Debt Rs.Cr.        1004 non-null   float64
 5   NP Qtr Rs.Cr.      992 non-null    float64
 6   6mth return %      960 non-null    float64
 7   1Yr return %       917 non-null    float64
 8   3Yrs return %      781 non-null    float64
 9   All time high Rs.  1004 non-null   float64
 10  All time low Rs.   1004 non-null   float64
 11  P/E                1004 non-null   float64
 12  EPS 12M Rs.        1004 non-null   float64
 13  Vol 1d             1004 non-null   int64  
 14  IV Rs.             1004 non-null   float64
 15  PB X PE            795 non-null    float64
 16  CMP / BV           923 non-nu

### handling null values in NP Qtr Rs.Cr.

In [110]:
merged_df[merged_df["NP Qtr Rs.Cr."].isnull()]

,S.No.,Name,CMP Rs.,Prom. Hold. %,Debt Rs.Cr.,NP Qtr Rs.Cr.,6mth return %,1Yr return %,3Yrs return %,All time high Rs.,All time low Rs.,P/E,EPS 12M Rs.,Vol 1d,IV Rs.,PB X PE,CMP / BV,Mar Cap Rs.Cr.,Sector
28,29.0,C2C Advanced,445.00,41.35,1.12,NaN,NaN,NaN,NaN,954.00,250.15,59.72,10.12,28200,604.3664,NaN,NaN,740.54,Aerospace & Defence
94,24.0,Delta Auto.,71.00,70.73,3.87,NaN,NaN,NaN,NaN,183.75,66.50,15.87,221.93,17000,3522.0291,NaN,NaN,108.56,Automobile
130,35.0,Chandan Healthca,173.95,49.43,42.01,NaN,NaN,NaN,NaN,215.40,140.00,26.13,8.14,45600,212.6982,NaN,NaN,425.34,Healthcare
203,26.0,Kabra Jewels,148.80,62.56,89.96,NaN,NaN,NaN,NaN,265.00,108.05,16.62,204.35,42000,3396.2970,NaN,NaN,156.02,"Diamond, Gems and Jewellery"
215,38.0,Manoj Jewellers,48.00,63.27,15.38,NaN,NaN,NaN,NaN,53.95,46.12,13.52,5.33,44000,72.0616,NaN,NaN,43.13,"Diamond, Gems and Jewellery"
222,45.0,Kenrik Indust.,22.47,72.01,0.78,NaN,NaN,NaN,NaN,27.57,22.47,26.50,1.18,24000,31.2700,NaN,NaN,28.09,"Diamond, Gems and Jewellery"
417,43.0,Srigee DLM,240.04,45.23,2.28,NaN,NaN,NaN,NaN,240.04,188.10,48.28,6.97,27600,336.5116,NaN,NaN,143.39,Plastic Products
429,55.0,Indobell Insul.,115.20,65.00,5.27,NaN,NaN,NaN,NaN,166.75,83.03,70.46,2.52,1500,177.5592,NaN,NaN,72.57,Plastic Products
430,56.0,Malpani Pipes,67.21,73.28,30.29,NaN,NaN,NaN,NaN,90.00,54.90,9.79,30.45,1600,298.1055,NaN,NaN,72.44,Plastic Products
448,74.0,L.K.Mehta Poly.,55.80,72.92,6.48,NaN,NaN,NaN,NaN,71.10,47.90,24.92,13.76,1600,342.8992,NaN,NaN,21.43,Plastic Products


In [111]:
null_percentage = merged_df["NP Qtr Rs.Cr."].isnull().mean() * 100
print(f"Percentage of null values in 'NP Qtr Rs.Cr.': {null_percentage:.2f}%")

Percentage of null values in 'NP Qtr Rs.Cr.': 1.20%


### ***we cannot calculate np qrt without using the columns revenue, tax, interest and also company statement, and also the percentage of null values is very less 1.19 so we are dropping these rows***

In [112]:
merged_df = merged_df.dropna(subset=["NP Qtr Rs.Cr."])

In [113]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 992 entries, 0 to 1006
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   S.No.              992 non-null    float64
 1   Name               992 non-null    object 
 2   CMP Rs.            992 non-null    float64
 3   Prom. Hold. %      992 non-null    float64
 4   Debt Rs.Cr.        992 non-null    float64
 5   NP Qtr Rs.Cr.      992 non-null    float64
 6   6mth return %      960 non-null    float64
 7   1Yr return %       917 non-null    float64
 8   3Yrs return %      781 non-null    float64
 9   All time high Rs.  992 non-null    float64
 10  All time low Rs.   992 non-null    float64
 11  P/E                992 non-null    float64
 12  EPS 12M Rs.        992 non-null    float64
 13  Vol 1d             992 non-null    int64  
 14  IV Rs.             992 non-null    float64
 15  PB X PE            795 non-null    float64
 16  CMP / BV           923 non-nul

### handling null values in NP Qtr Rs.Cr.

In [114]:
null_rows = merged_df[merged_df["6mth return %"].isnull() & merged_df["1Yr return %"].isnull() & merged_df["3Yrs return %"].isnull()]
null_rows

,S.No.,Name,CMP Rs.,Prom. Hold. %,Debt Rs.Cr.,NP Qtr Rs.Cr.,6mth return %,1Yr return %,3Yrs return %,All time high Rs.,All time low Rs.,P/E,EPS 12M Rs.,Vol 1d,IV Rs.,PB X PE,CMP / BV,Mar Cap Rs.Cr.,Sector
15,16.0,Unimech Aero.,1096.70,79.82,29.90,15.57,NaN,NaN,NaN,1523.75,850.00,96.070000,13.210000,478661,1269.084700,NaN,NaN,5576.93,Aerospace & Defence
24,25.0,Rossell Techsys,347.70,74.80,195.57,5.20,NaN,NaN,NaN,579.00,231.15,115.960000,2242.000000,8971,259982.320000,1250.05,10.78,1299.95,Aerospace & Defence
83,13.0,Ather Energy,315.05,42.09,619.30,-234.40,NaN,NaN,NaN,333.00,287.30,-11.271914,-27.950000,3046053,315.050000,NaN,NaN,11732.42,Automobile
90,20.0,Indo Farm Equip.,164.00,69.44,248.67,3.98,NaN,NaN,NaN,293.20,132.60,47.460000,4.150000,128751,196.959000,NaN,NaN,788.79,Automobile
108,13.0,Dr Agarwal's Hea,394.05,32.45,966.43,28.24,NaN,NaN,NaN,464.00,330.00,182.150000,3.574233,221724,651.046521,NaN,NaN,12453.55,Healthcare
120,25.0,Laxmi Dental,400.25,41.70,61.39,4.82,NaN,NaN,NaN,584.00,307.55,87.870000,804.810000,53485,70718.654700,NaN,NaN,2196.33,Healthcare
125,30.0,Suraksha Diagno.,303.50,48.78,85.47,6.36,NaN,NaN,NaN,449.00,231.00,59.800000,353.330000,36851,449.730000,473.62,7.92,1582.22,Healthcare
152,57.0,Shanmuga Hos.,37.50,72.11,9.76,2.39,NaN,NaN,NaN,54.00,31.00,9.710000,5.370000,22000,52.142700,NaN,NaN,51.05,Healthcare
213,36.0,Divine Hira,44.90,72.70,12.93,2.50,NaN,NaN,NaN,90.00,43.25,39.580000,7.664732,6400,303.370092,NaN,NaN,58.58,"Diamond, Gems and Jewellery"
218,41.0,Equilateral Ent.,28.96,8.04,0.00,0.15,NaN,NaN,NaN,137.00,7.27,223.210000,0.130000,1343,10.670000,736.59,3.30,35.71,"Diamond, Gems and Jewellery"


In [115]:
null_count = len(null_rows)
total_count = len(merged_df)
percentage = (null_count / total_count) * 100
percentage

2.9233870967741935

### ***Because the companies which did not publish their profits(6mth return %", "1Yr return %", "3Yrs return %), so we dropped the rows since we cannot calculate profits without these columns also the percentage is just 2.920443101711984***

In [116]:
merged_df = merged_df.dropna(subset=["6mth return %", "1Yr return %", "3Yrs return %"])

In [117]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 777 entries, 0 to 1006
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   S.No.              777 non-null    float64
 1   Name               777 non-null    object 
 2   CMP Rs.            777 non-null    float64
 3   Prom. Hold. %      777 non-null    float64
 4   Debt Rs.Cr.        777 non-null    float64
 5   NP Qtr Rs.Cr.      777 non-null    float64
 6   6mth return %      777 non-null    float64
 7   1Yr return %       777 non-null    float64
 8   3Yrs return %      777 non-null    float64
 9   All time high Rs.  777 non-null    float64
 10  All time low Rs.   777 non-null    float64
 11  P/E                777 non-null    float64
 12  EPS 12M Rs.        777 non-null    float64
 13  Vol 1d             777 non-null    int64  
 14  IV Rs.             777 non-null    float64
 15  PB X PE            626 non-null    float64
 16  CMP / BV           731 non-nul

### handling null values in PB * PE

In [118]:
merged_df.loc[
    merged_df["PB X PE"].isnull() &
    merged_df["CMP / BV"].notnull() &
    merged_df["P/E"].notnull(),
    "PB X PE"
] = merged_df["CMP / BV"] * merged_df["P/E"]

In [119]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 777 entries, 0 to 1006
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   S.No.              777 non-null    float64
 1   Name               777 non-null    object 
 2   CMP Rs.            777 non-null    float64
 3   Prom. Hold. %      777 non-null    float64
 4   Debt Rs.Cr.        777 non-null    float64
 5   NP Qtr Rs.Cr.      777 non-null    float64
 6   6mth return %      777 non-null    float64
 7   1Yr return %       777 non-null    float64
 8   3Yrs return %      777 non-null    float64
 9   All time high Rs.  777 non-null    float64
 10  All time low Rs.   777 non-null    float64
 11  P/E                777 non-null    float64
 12  EPS 12M Rs.        777 non-null    float64
 13  Vol 1d             777 non-null    int64  
 14  IV Rs.             777 non-null    float64
 15  PB X PE            731 non-null    float64
 16  CMP / BV           731 non-nul

In [125]:
total_rows = len(merged_df)
pbxpe_null_percent = merged_df['PB X PE'].isna().sum() / total_rows * 100
cmp_bv_null_percent = merged_df['CMP / BV'].isna().sum() / total_rows * 100

print(f"PB X PE - Null %: {pbxpe_null_percent:.2f}%")
print(f"CMP / BV - Null %: {cmp_bv_null_percent:.2f}%")


PB X PE - Null %: 5.92%
CMP / BV - Null %: 5.92%


### ***if both PB X PE and CMP / BV are missing, and you don’t have Book Value, P/B ratio, or number of shares —there is no pure formula using your current columns that can fill these values and the percentage is only 5.92%, so we are droping these rows.****

In [126]:
merged_df = merged_df.dropna(subset=["PB X PE", "CMP / BV"])

In [127]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 731 entries, 0 to 1005
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   S.No.              731 non-null    float64
 1   Name               731 non-null    object 
 2   CMP Rs.            731 non-null    float64
 3   Prom. Hold. %      731 non-null    float64
 4   Debt Rs.Cr.        731 non-null    float64
 5   NP Qtr Rs.Cr.      731 non-null    float64
 6   6mth return %      731 non-null    float64
 7   1Yr return %       731 non-null    float64
 8   3Yrs return %      731 non-null    float64
 9   All time high Rs.  731 non-null    float64
 10  All time low Rs.   731 non-null    float64
 11  P/E                731 non-null    float64
 12  EPS 12M Rs.        731 non-null    float64
 13  Vol 1d             731 non-null    int64  
 14  IV Rs.             731 non-null    float64
 15  PB X PE            731 non-null    float64
 16  CMP / BV           731 non-nul

In [128]:
merged_df.to_csv("final.csv")

In [4]:
df = pd.read_csv('final.csv')

# ***************************************************************

# Finding Insights of the data

## ***How to choose a Long-Term stock***

In [5]:
# Filter for long-term growth stocks
long_term = df[
    (df['P/E'] > 10) & (df['P/E'] < 35) &
    (df['EPS 12M Rs.'] > 10) &
    (df['3Yrs return %'] > 50) &
    (df['Prom. Hold. %'] > 50) &
    (df['Debt Rs.Cr.'] < 1000)
]

# Show top 10 individual stocks
long_term_display = long_term[['Name', 'Sector', 'P/E', 'EPS 12M Rs.', '3Yrs return %', 'Prom. Hold. %', 'Debt Rs.Cr.']]

# print("Top 10 Long-Term Stocks:\n", long_term_display.sort_values(by='3Yrs return %', ascending=False).head(10))

# Group long-term stocks by Sector
long_term_sector = long_term.groupby('Sector').agg({
    'Name': 'count',
    '3Yrs return %': 'mean',
    'EPS 12M Rs.': 'mean',
    'Debt Rs.Cr.': 'mean'
}).rename(columns={'Name': 'Stock Count'}).sort_values(by='3Yrs return %', ascending=False)


# Display sector summaries
print("\n---Sector Performance ---")
long_term_sector



---Sector Performance ---


,Stock Count,3Yrs return %,EPS 12M Rs.,Debt Rs.Cr.
Sector,,,,
Automobile,1,119.290000,607.710000,17.430000
Steel,4,79.240000,15.900000,121.637500
Healthcare,2,73.900000,102.565000,226.300000
IT-Software,4,62.742500,53.907500,115.090000
Pharmaceuticals,2,62.220000,70.180000,9.480000
Plastic Products,4,60.087500,21.922500,223.332500
"Diamond, Gems and Jewellery",3,52.636667,25.416667,281.253333


## ***How to choose a Short-Term stock***

In [6]:
# Filter for short-term momentum stocks
short_term = df[
    ((df['6mth return %'] > 30) | (df['1Yr return %'] > 50)) &
    (df['Vol 1d'] > 100000)  # Filter for decent trading volume
]

# Show selected columns
short_term_display = short_term[['Name', 'Sector', '6mth return %', '1Yr return %', 'Vol 1d']]

# Group by Sector for summary
short_term_sector = short_term.groupby('Sector').agg({
    'Name': 'count',
    '6mth return %': 'mean',
    '1Yr return %': 'mean',
    'Vol 1d': 'mean'
}).rename(columns={'Name': 'Stock Count'}).sort_values(by='6mth return %', ascending=False)

# Convert Vol 1d to lakhs for readability
short_term_sector['Vol 1d (Lakhs)'] = short_term_sector['Vol 1d'] / 1e5

# Display results
print("\n---Top Short-Term Performing Sectors---")
display(short_term_sector)

# Optional: See top 10 short-term stocks
short_term_display.sort_values(by='6mth return %', ascending=False).head(10)


---Top Short-Term Performing Sectors---


,Stock Count,6mth return %,1Yr return %,Vol 1d,Vol 1d (Lakhs)
Sector,,,,,
Steel,6,182.398333,209.758333,3.724995e+05,3.724995
Automobile,1,58.080000,21.890000,1.114720e+05,1.114720
Aerospace & Defence,16,49.910625,88.541250,1.214673e+07,121.467307
Banks,2,34.300000,-14.370000,1.073318e+07,107.331810
Healthcare,8,28.348750,71.760000,4.941792e+05,4.941793
Pharmaceuticals,14,18.368571,88.780714,7.647786e+05,7.647786
IT-Software,9,15.631111,79.955556,5.556367e+05,5.556367
Consumer Durables,6,11.200000,114.520000,5.515858e+05,5.515858
"Diamond, Gems and Jewellery",4,0.287500,133.467500,3.367284e+06,33.672845


,Name,Sector,6mth return %,1Yr return %,Vol 1d
239,Rajas. Tube Mfg,Steel,980.75,980.75,376271
409,Shukra Pharma.,Pharmaceuticals,120.00,25.17,530053
4,Bharat Dynamics,Aerospace & Defence,86.15,78.01,11198564
13,Paras Defence,Aerospace & Defence,83.51,149.07,8254617
589,AXISCADES Tech.,IT-Software,83.30,54.57,153407
2,Mazagon Dock,Aerospace & Defence,79.44,188.74,17647411
19,Sika Interplant,Aerospace & Defence,77.97,77.63,148453
6,Garden Reach Sh.,Aerospace & Defence,76.46,150.45,15742150
18,Jaykay Enter.,Aerospace & Defence,58.75,128.43,522662
68,Force Motors,Automobile,58.08,21.89,111472


## ***How to understand if a stock is undervalued or overvalued***

In [9]:
import pandas as pd

# Load data
df = pd.read_csv("final.csv")

# Evaluate Valuation
def evaluate_valuation(row):
    if row['CMP Rs.'] < row['IV Rs.']:
        return 'Undervalued'
    elif row['CMP Rs.'] > row['IV Rs.']:
        return 'Overvalued'
    else:
        return 'Fairly valued'


# Create new column for valuation
df['Valuation Status'] = df.apply(evaluate_valuation, axis=1)

# Group by Sector and Valuation Status
sector_valuation_summary = df.groupby(['Sector', 'Valuation Status']).size().unstack(fill_value=0)

# Optional: Add a Total column
sector_valuation_summary['Total Stocks'] = sector_valuation_summary.sum(axis=1)

# Sort by number of undervalued stocks
sector_valuation_summary = sector_valuation_summary.sort_values(by='Undervalued', ascending=False)
sector_valuation_summary = sector_valuation_summary.reset_index()
sector_valuation_summary.columns.name = None 
# Display the result
print("\n--- Sector-wise Valuation Summary ---")
display(sector_valuation_summary)


--- Sector-wise Valuation Summary ---


,Sector,Fairly valued,Overvalued,Undervalued,Total Stocks
0,IT-Software,29,118,35,182
1,Pharmaceuticals,26,115,27,168
2,Banks,0,14,22,36
3,Steel,11,60,21,92
4,Plastic Products,7,46,10,63
5,Healthcare,8,37,9,54
6,"Diamond, Gems and Jewellery",8,31,5,44
7,Consumer Durables,6,41,4,51
8,Automobile,0,17,1,18
9,Aerospace & Defence,1,22,0,23


## ***Do you recommend in investing blue chip stocks first? why or why not?***

In [21]:
import pandas as pd

# Load the CSV
df = pd.read_csv('final.csv')

# View all column names (uncomment for debugging)
# print(df.columns.tolist())

# Convert necessary columns to numeric
df['Mar Cap Rs.Cr.'] = pd.to_numeric(df['Mar Cap Rs.Cr.'], errors='coerce')
df['EPS 12M Rs.'] = pd.to_numeric(df['EPS 12M Rs.'], errors='coerce')

# Define blue-chip criteria
blue_chip_stocks = df[
    (df['Mar Cap Rs.Cr.'] >= 20000) &  # Large Market Cap (20,000+ Cr)
    (df['EPS 12M Rs.'] > 20)           # Good earnings
]

# Group sector-wise summary
sector_bluechip_summary = blue_chip_stocks.groupby('Sector').agg({
    'Name': 'count',
    'Mar Cap Rs.Cr.': 'mean',
    'EPS 12M Rs.': 'mean',
    'P/E': 'mean',
    'CMP Rs.': 'mean'
}).rename(columns={'Name': 'Stock Count'})

# Display
print("\n--- Sector-wise Blue Chip Stock Summary ---")
sector_bluechip_summary.reset_index()


--- Sector-wise Blue Chip Stock Summary ---


,Sector,Stock Count,Mar Cap Rs.Cr.,EPS 12M Rs.,P/E,CMP Rs.
0,Aerospace & Defence,5,138772.902000,79.074000,64.618000,5436.320000
1,Automobile,8,214500.750000,182.791250,31.156250,5195.168750
2,Banks,11,407759.043636,67.480000,12.930000,912.745455
3,Consumer Durables,5,58912.502000,55.088000,85.506000,5488.390000
4,"Diamond, Gems and Jewellery",1,322364.560000,37.590000,96.600000,3634.900000
5,Healthcare,5,42842.508000,65.350000,69.004000,2918.340000
6,IT-Software,13,240386.886923,103.094615,41.034615,4051.176923
7,Pharmaceuticals,16,92839.681875,107.044375,37.548750,4262.684375
8,Plastic Products,1,46794.140000,75.640000,48.700000,3683.900000
9,Steel,6,51630.450000,34.828333,40.796667,1364.808333


In [21]:
df.columns

Index(['Unnamed: 0', 'S.No.', 'Name', 'CMP Rs.', 'Prom. Hold. %',
       'Debt Rs.Cr.', 'NP Qtr Rs.Cr.', '6mth return %', '1Yr return %',
       '3Yrs return %', 'All time high Rs.', 'All time low Rs.', 'P/E',
       'EPS 12M Rs.', 'Vol 1d', 'IV Rs.', 'PB X PE', 'CMP / BV',
       'Mar Cap Rs.Cr.', 'Sector'],
      dtype='object')

## ***Volatility***

In [23]:
import pandas as pd

# Load dataset
df = pd.read_csv("final.csv")
df.columns = df.columns.str.encode('ascii', 'ignore').str.decode('ascii').str.strip()

# Rename columns
df = df.rename(columns={
    'Name': 'Stock',
    'CMP Rs.': 'Current Price',
    'All time high Rs.': 'All Time High',
    'All time low Rs.': 'All Time Low',
    '1Yr return %': '1Y Return',
    '3Yrs return %': '3Y Return'
})

# Drop missing and filter out zero/near-zero all-time lows
df_clean = df.dropna(subset=['Sector', 'Stock', 'Current Price', 'All Time High', 'All Time Low', '1Y Return', '3Y Return'])
df_clean = df_clean[df_clean['All Time Low'] > 1]  # Prevent abnormal Gain from Low %

# Calculate % changes
df_clean['Drop from High %'] = ((df_clean['All Time High'] - df_clean['Current Price']) / df_clean['All Time High']) * 100
df_clean['Gain from Low %'] = ((df_clean['Current Price'] - df_clean['All Time Low']) / df_clean['All Time Low']) * 100

# Group by sector
sector_volatility = df_clean.groupby('Sector').agg({
    'Stock': 'count',
    'Drop from High %': 'mean',
    'Gain from Low %': 'mean',
    '1Y Return': 'mean',
    '3Y Return': 'mean'
}).rename(columns={'Stock': 'Stock Count'})

# Display cleaned output
sector_volatility.reset_index()


,Sector,Stock Count,Drop from High %,Gain from Low %,1Y Return,3Y Return
0,Aerospace & Defence,21,22.560485,10333.274113,59.992381,85.465714
1,Automobile,16,28.273337,10819.150473,-3.391250,33.457500
2,Banks,36,43.462172,1109.766669,-7.288333,27.512778
3,Consumer Durables,48,42.824034,8373.167634,17.930000,22.016250
4,"Diamond, Gems and Jewellery",33,53.102813,3507.054055,22.928485,30.643333
5,Healthcare,46,38.966088,5560.832409,24.837174,23.951957
6,IT-Software,152,48.705432,3938.121325,19.825526,28.033026
7,Pharmaceuticals,148,40.364710,5301.425127,16.920878,20.240068
8,Plastic Products,51,47.767667,9430.695963,17.966078,27.106078
9,Steel,80,44.957699,4002.949894,-1.262000,28.378750


## ***Risky stocks***

In [51]:
import pandas as pd

# Load the data
df = pd.read_csv("final.csv")

# Clean and convert necessary columns
cols_to_numeric = ['CMP Rs.', 'All time high Rs.', 'All time low Rs.',
                   '1Yr return %', '3Yrs return %', 'EPS 12M Rs.', 'P/E', 'Debt Rs.Cr.']
for col in cols_to_numeric:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Calculate drop from high and gain from low
df['Drop from High %'] = 100 * (df['All time high Rs.'] - df['CMP Rs.']) / df['All time high Rs.']
df['Gain from Low %'] = 100 * (df['CMP Rs.'] - df['All time low Rs.']) / df['All time low Rs.']

# Define risk conditions
df['High Volatility'] = (df['Drop from High %'] > 80) & (df['Gain from Low %'] > 200)
df['Poor Returns'] = (df['1Yr return %'] < 0) | (df['3Yrs return %'] < 0)
df['Weak Fundamentals'] = (df['EPS 12M Rs.'] < 1) | (df['P/E'] > 40) | (df['Debt Rs.Cr.'] > 500)

# Risk score (0 to 3)
df['Risk Score'] = df[['High Volatility', 'Poor Returns', 'Weak Fundamentals']].sum(axis=1)

# Assign Risk Level with new naming
df['Risk Level'] = df['Risk Score'].map({
    0: 'No Risk',
    1: 'Low Risk',
    2: 'Moderate Risk',
    3: 'High Risk'
})

# Group by Sector and Risk Level
sector_risk_summary = df.groupby(['Sector', 'Risk Level']).size().unstack(fill_value=0)

# Add total stocks per sector
sector_risk_summary['Total Stocks'] = sector_risk_summary.sum(axis=1)

# Safe stocks = No Risk + Low Risk
sector_risk_summary['Safe Stocks'] = sector_risk_summary.get('No Risk', 0) + sector_risk_summary.get('Low Risk', 0)

# Unsafe stocks = Moderate Risk + High Risk
sector_risk_summary['Unsafe Stocks'] = sector_risk_summary.get('Moderate Risk', 0) + sector_risk_summary.get('High Risk', 0)

# Calculate Safe % and Unsafe % per sector
sector_risk_summary['Safe %'] = 100 * sector_risk_summary['Safe Stocks'] / sector_risk_summary['Total Stocks']
sector_risk_summary['Unsafe %'] = 100 * sector_risk_summary['Unsafe Stocks'] / sector_risk_summary['Total Stocks']

# Sort by Unsafe % descending (most risky first)
sector_risk_summary = sector_risk_summary.sort_values(by='Unsafe %', ascending=False)

# Reset index for nicer display
sector_risk_summary = sector_risk_summary.reset_index()

# Show the result
print("\n--- Sector-wise Risk Level Summary ---")
sector_risk_summary




--- Sector-wise Risk Level Summary ---


Risk Level,Sector,High Risk,Low Risk,Moderate Risk,No Risk,Total Stocks,Safe Stocks,Unsafe Stocks,Safe %,Unsafe %
0,Banks,5,12,19,0,36,12,24,33.333333,66.666667
1,"Diamond, Gems and Jewellery",5,15,18,6,44,21,23,47.727273,52.272727
2,Automobile,0,6,8,4,18,10,8,55.555556,44.444444
3,Consumer Durables,2,22,20,7,51,29,22,56.862745,43.137255
4,IT-Software,23,95,43,21,182,116,66,63.736264,36.263736
5,Steel,1,51,30,10,92,61,31,66.304348,33.695652
6,Plastic Products,2,29,19,13,63,42,21,66.666667,33.333333
7,Healthcare,5,27,12,10,54,37,17,68.518519,31.481481
8,Aerospace & Defence,0,16,7,0,23,16,7,69.565217,30.434783
9,Pharmaceuticals,7,88,38,35,168,123,45,73.214286,26.785714


## ***Sector wise Return Projection***

In [12]:
sector_returns = []

for sector, group in df.groupby('Sector'):
    one_year_avg = group['1Yr return %'].mean(skipna=True)
    three_year_cagr = ((group['3Yrs return %'] / 100 + 1) ** (1/3) - 1).mean(skipna=True) * 100

    if pd.notnull(one_year_avg) and pd.notnull(three_year_cagr):
        est_return = (one_year_avg + three_year_cagr) / 2
    elif pd.notnull(three_year_cagr):
        est_return = three_year_cagr
    else:
        est_return = one_year_avg

    r = est_return / 100
    projection = {
        "Sector": sector,
        "Estimated Annual Return %": round(est_return, 2),
        "5 Year Return (₹100)": round(100 * ((1 + r) ** 5), 2)
    }
    sector_returns.append(projection)

# Convert to DataFrame
sector_return_df = pd.DataFrame(sector_returns).sort_values(by="Estimated Annual Return %", ascending=False)
sector_return_df


,Sector,Estimated Annual Return %,5 Year Return (₹100)
0,Aerospace & Defence,41.34,564.09
6,IT-Software,14.85,199.85
5,Healthcare,12.91,183.49
4,"Diamond, Gems and Jewellery",12.82,182.78
3,Consumer Durables,11.43,171.82
8,Plastic Products,10.75,166.60
7,Pharmaceuticals,9.85,159.97
9,Steel,8.18,148.16
1,Automobile,3.27,117.45
2,Banks,0.42,102.13


## ***Suspect stocks (Pump & Dump) analysis sector-wise***

In [10]:
import pandas as pd

# Load the data
df = pd.read_csv('final.csv')

# Convert necessary columns to numeric
cols_to_check = ['CMP Rs.', 'All time high Rs.', 'All time low Rs.', '1Yr return %', '3Yrs return %',
                 'EPS 12M Rs.', 'P/E', 'Debt Rs.Cr.']
df[cols_to_check] = df[cols_to_check].apply(pd.to_numeric, errors='coerce')

# Calculate drop from high and gain from low
df['Drop from High %'] = 100 * (df['All time high Rs.'] - df['CMP Rs.']) / df['All time high Rs.']
df['Gain from Low %'] = 100 * (df['CMP Rs.'] - df['All time low Rs.']) / df['All time low Rs.']

# Define suspicious flags
df['Extreme Volatility'] = (df['Drop from High %'] > 80) & (df['Gain from Low %'] > 300)
df['Very High PE'] = df['P/E'] > 60
df['Low EPS'] = df['EPS 12M Rs.'] < 1
df['High Debt'] = df['Debt Rs.Cr.'] > 500
df['Consistent Negative Returns'] = (df['1Yr return %'] < 0) & (df['3Yrs return %'] < 0)

# Combine all red flags
df['P&D Suspect Score'] = df[[
    'Extreme Volatility', 'Very High PE', 'Low EPS', 'High Debt', 'Consistent Negative Returns'
]].sum(axis=1)

# Label warnings
df['P&D Warning'] = df['P&D Suspect Score'] >= 3

# 🔍 Sector-wise suspect summary
suspect_summary = df.groupby('Sector')['P&D Warning'].value_counts().unstack(fill_value=0)
suspect_summary['Total Stocks'] = suspect_summary.sum(axis=1)
suspect_summary['% Suspect'] = round((suspect_summary[True] / suspect_summary['Total Stocks']) * 100, 2)
suspect_summary = suspect_summary.sort_values('% Suspect', ascending=False)

# 📊 Show summary
print("\n Sector-wise Suspect Summary:\n")
print(suspect_summary)

# 📋 Show detailed suspect stocks
suspect_stocks = df[df['P&D Warning']][[
    'Sector', 'Name', 'CMP Rs.', '1Yr return %', '3Yrs return %', 'P/E', 'EPS 12M Rs.',
    'Debt Rs.Cr.', 'Drop from High %', 'Gain from Low %', 'P&D Suspect Score'
]]

suspect_stocks = suspect_stocks.sort_values(by='P&D Suspect Score', ascending=False)

print("\n Suspect Stocks:\n")
suspect_stocks



 Sector-wise Suspect Summary:

P&D Warning                  False  True  Total Stocks  % Suspect
Sector                                                           
Diamond, Gems and Jewellery     39     5            44      11.36
IT-Software                    168    14           182       7.69
Healthcare                      50     4            54       7.41
Plastic Products                59     4            63       6.35
Automobile                      17     1            18       5.56
Pharmaceuticals                160     8           168       4.76
Consumer Durables               49     2            51       3.92
Steel                           91     1            92       1.09
Aerospace & Defence             23     0            23       0.00
Banks                           36     0            36       0.00

 Suspect Stocks:



,Sector,Name,CMP Rs.,1Yr return %,3Yrs return %,P/E,EPS 12M Rs.,Debt Rs.Cr.,Drop from High %,Gain from Low %,P&D Suspect Score
462,Pharmaceuticals,Biofil Chemicals,46.60,-32.68,-7.43,124.540000,0.37,0.46,86.913788,3.457252e+03,4
461,Pharmaceuticals,Rajnish Wellness,0.98,-85.63,-45.81,94.140000,0.01,0.30,95.219512,5.533333e+02,4
124,Healthcare,Global Longlife,22.00,-47.90,-37.46,115.500000,0.19,0.00,84.408221,inf,4
134,"Diamond, Gems and Jewellery",Rajesh Exports,203.50,-32.46,-30.34,157.020000,1.30,735.91,80.236962,1.024309e+03,4
611,IT-Software,Vakrangee,10.50,-58.45,-28.75,177.640000,0.06,4.59,97.748424,2.400000e+03,4
627,IT-Software,DigiSpice Tech.,22.73,-17.30,-9.82,72.080000,0.46,61.29,85.050970,7.117857e+02,4
633,IT-Software,FCS Software,2.70,-29.87,-5.51,49.400000,0.02,0.00,82.524272,5.300000e+03,3
511,Consumer Durables,Sheela Foam,654.55,-29.39,-24.87,89.030000,8.84,1462.36,67.716559,5.399365e+01,3
548,Consumer Durables,Milestone Furn.,4.41,-36.91,-20.99,-6.485294,-0.68,0.42,90.297030,inf,3
618,IT-Software,Subex,14.20,-51.37,-23.83,-25.357143,-0.56,19.38,98.232182,4.071429e+02,3


## ***Fundamental Rating of companies sector wise***

In [7]:
import pandas as pd

# Load the data
df = pd.read_csv('final.csv')

# Define the function to evaluate fundamentals
def evaluate_fundamentals(row):
    score = 0

    # Promoter Holding > 50%
    if row['Prom. Hold. %'] > 50:
        score += 1

    # Debt < 1000 Cr
    if row['Debt Rs.Cr.'] < 1000:
        score += 1

    # Net Profit > 0
    if row['NP Qtr Rs.Cr.'] > 0:
        score += 1

    # EPS > 5
    if row['EPS 12M Rs.'] > 5:
        score += 1

    # P/E < 25
    if row['P/E'] < 25:
        score += 1

    # Final Rating
    if score >= 4:
        return "Strong"
    elif score >= 2:
        return "Moderate"
    else:
        return "Weak"

# Apply evaluation sector-wise
df['Fundamental Rating'] = df.groupby('Sector').apply(
    lambda g: g.apply(evaluate_fundamentals, axis=1)
).reset_index(level=0, drop=True)

# Display evaluated data
result = df[['Sector', 'Name', 'Fundamental Rating']]
# print("\nCompany Ratings:\n")
# print(result)

# Create summary table
summary = df.groupby(['Sector', 'Fundamental Rating']).size().unstack(fill_value=0)

# Add Total and % Strong
summary['Total'] = summary.sum(axis=1)
summary['% Strong'] = round(summary['Strong'] / summary['Total'] * 100, 2)

# Sort by % Strong
summary = summary.sort_values('% Strong', ascending=False)

# Show summary
print("\nSector-wise Summary:\n")
summary



Sector-wise Summary:



C:\Users\lenovo\AppData\Local\Temp\ipykernel_9724\1880158479.py:39: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df['Fundamental Rating'] = df.groupby('Sector').apply(


Fundamental Rating,Moderate,Strong,Weak,Total,% Strong
Sector,,,,,
Aerospace & Defence,10,13,0,23,56.52
Steel,41,50,1,92,54.35
Plastic Products,33,29,1,63,46.03
Pharmaceuticals,87,75,6,168,44.64
Automobile,11,7,0,18,38.89
Consumer Durables,31,19,1,51,37.25
IT-Software,115,66,1,182,36.26
Healthcare,35,18,1,54,33.33
"Diamond, Gems and Jewellery",29,14,1,44,31.82
